
1. **CSV File Upload**:
   - Place the CSV file (e.g., `output_file.csv`) in a directory accessible to Neo4j (e.g., `/import`).
   - Update the `neo4j.conf` file to include:
     ```
     dbms.directories.import=/path/to/your/import/directory
     ```

2. **Neo4j Server**:
   - Ensure the Neo4j server is running and accessible.
   - Use the correct URI, username, and password for authentication.

3. **Python Environment**:
   - Install the required libraries:
     ```bash
     pip install neo4j pandas
     ```


In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Connection details
URI = "neo4j://127.0.0.1:7687"
AUTH = ("neo4j", "12345678")

# Read CSV file
csv_path = r"C:\Users\Priyam\Documents\Temp\Priyam\RM\GoodCSV.csv"
df = pd.read_csv(csv_path)

# Clean up the dataframe (remove empty columns)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

try:
    # Establish connection to the Neo4j database
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            print(f"Starting CSV data load into Neo4j... ({len(df)} rows)")
            
            # Load data row by row
            for index, row in df.iterrows():
                subject = row['Subject']
                relationship = row['Relationship']
                obj = row['Object'] if pd.notna(row['Object']) else 'None'
                
                query = """
                MERGE (t:SubjectNode { name: $subject })
                MERGE (o:ObjectNode { name: $object })
                MERGE (t)-[r:RELATIONSHIP { name: $relationship }]-(o)
                """
                
                session.run(query, subject=subject, relationship=relationship.upper(), object=obj)
                
                if (index + 1) % 100 == 0:
                    print(f"Processed {index + 1} rows...")
            
            print("CSV data has been successfully loaded into Neo4j!")

            # Verify the number of nodes and relationships loaded
            query_nodes = "MATCH (n) RETURN COUNT(n) AS TotalNodes;"
            query_relationships = "MATCH ()-[r]->() RETURN COUNT(r) AS TotalRelationships;"

            result_nodes = session.run(query_nodes)
            for record in result_nodes:
                print("Total Nodes in the Database:", record["TotalNodes"])

            result_relationships = session.run(query_relationships)
            for record in result_relationships:
                print("Total Relationships in the Database:", record["TotalRelationships"])

except Exception as e:
    print(f"An error occurred: {e}")

In [1]:
!pip install neo4j

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
